<a href="https://colab.research.google.com/github/YoelCanaza/curso-transfer-learning-huggingface/blob/main/my-practice_Computer_vision_con_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visión por computadora con Hugging Face

Así como los modelos basados ​​en Transformers han revolucionado el PLN, ahora vemos una explosión de investigaciones que los aplican a todo tipo de dominios.

Uno de los más revolucionarios fue el Vision Transformer (ViT), que fue presentado en junio de 2021 por un equipo de investigadores de Google Brain.

Usaremos el `Trainer` de Hugging Face para entrenar nuestro modelo de clasificación de imágenes.

Pero antes tenemos que proporcionarle información sobre nuestros datos, el procesamiento de los datos, el modelo, las métricas e información del entrenamiento.

En específico necesitamos definir cada uno de estos puntos:

    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor



## Procesando los datos para visión



Usaremos un Dataset del Hub y lo usaremos para afinar un ViT preentrenado con 🤗 `Transformers`.

### Descargando el dataset

Usaremos el dataset [beans](https://huggingface.co/datasets/beans). Con base en una imagen de una hoja de frijol buscamos predecir si la hoja está saludable o si está enferma. El tipo de enfermedad que tiene (Angular Leaf Spot o Bean Rust).

In [2]:
%%capture
#usamos capture para no tener que ver toda la descarga
!pip install datasets transformers

In [3]:
from datasets import load_dataset # con esto podremos descargar el dataset directamente desde el hub
ds_beans = load_dataset("beans")

Generating train split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/133 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/128 [00:00<?, ? examples/s]

Miremos un ejemplo.

Como en la key `image` tenemos una imagen tipo PIL entonces podemos verla.

Aprendamos un poco más sobre las etiquetas en nuestro Dataset. Por ejemplo, notamos que tenemos tres etiquetas.

El método `int2str` de una `ClassLabel` nos permite pasar la representación en número (entero) de la etiqueta y recibir el nombre de la clase.

### Cargando el ViT Feature Extractor

Prepararemos estas imágenes para nuestro modelo.

Cuando se entrenan los modelos ViT se aplican transformaciones específicas a las imágenes que se les alimentan. Si usa las transformaciones incorrectas en su imagen, el modelo no entenderá lo que está viendo. 🖼 ➡️

Para asegurarnos de que aplicamos las transformaciones correctas, usaremos un `ViTFeatureExtractor` inicializado con una configuración que se guardó junto con el modelo pre-entrenado que planeamos usar. En nuestro caso usaremos el modelo `google/vit-base-patch16-224-in21k`, así que carguemos su deacture extractor desde el Hub.

Un extractor de características se encarga de preparar las características de los inputs de un modelo.


Puedes ver la configuración del extractor imprimiéndola.

Para procesar una imagen, simplemente pásala a la función `call` del extractor. Esto devolverá un diccionario que contiene valores de pixeles, que es la representación numérica que se pasará al modelo.

De forma determinada obtenemos una matriz NumPy, pero si agregamos el argumento `return_tensors='pt'`, obtendremos tensores de PyTorch en su lugar.

Nos regresa un diccionario con una sola key.

Podemos ver la forma del tensor de pixeles.

### Procesando el dataset

Ahora podemos leer imágenes y transformarlas en inputs para nuestro modelo. Escribamos una función que juntará esos dos pasos. Recuerda que `feature_extractor` retorna un diccionario con la key `pixel_values`. Le sumamos una segunda key con las etiquetas `labels`.

Así se ve un ejemplo procesado de esta manera.

Podemos llamar a la función `map` y aplicar esto a todos los ejemplos a la vez. Pero esto puede ser muy lento, especialmente si usa un dataset más grande.

Entonces podemos aplicar una ***transformación*** al dataset. Las transformaciones solo se aplican a los ejemplos a medida que los indexamos.

Sin embargo, primero deberá actualizar la última función para aceptar un batch posiblemente con más de una imagen, ya que eso es lo que espera `ds.with_transform`.

Puedes aplicar esto directamente al dataset mediante `ds.with_transform(transform)`

Ahora, cada vez que obtengas un ejemplo del dataset, la transformación se aplicará en tiempo real.

El tensor `pixel_values` ​​resultante tendrá forma (2, 3, 224, 224) porque pasamos un batch de dos ejemplos.

### Definiendo el data collator


Los data collators, o recopiladores de datos, son objetos que forman batches utilizando una lista de ejemplos de nuestros datasets. Para poder generar los batches, los data collators pueden aplicar algún procesamiento (como padding en los ejemplos con texto).

Definimos una función, `collate_fn`, que fungirá como nuestro data collator. Devolverá un diccionario por cada batch. Recibirá un batch de datos que luego serán procesadas.

Los batches llegan como listas de dicts. Cada dict tiene los `label` y `pixel_values` de sus respectivos ejemplos, por lo que puedes simplemente desempaquetarlos y apilarlos en tensores de batches. `torch.stack` nos permite concatenar (pegar) tensores.


## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica

De la biblioteca `Datasets` también podemos cargar métricas. `accuracy` se puede usar fácilmente para comparar las predicciones con las etiquetas originales.

### Configurando `Trainer`

Carguemos el modelo preentrenado. Agregaremos `num_labels` para que el modelo cree un encabezado de clasificación con el número correcto de etiquetas. También incluiremos las asignaciones `id2label` y `label2id` para tener etiquetas legibles por humanos en el widget del Hub.

Lo último que se necesita antes de eso es establecer la configuración de entrenamiento definiendo `TrainingArguments`.

La mayoría de estos se explican por sí mismos, pero uno que es bastante importante aquí es `remove_unused_columns=False`. Este eliminará cualquier función que no utilice la función de llamada del modelo. De forma predeterminada es True porque, por lo general, es ideal eliminar las columnas de funciones no utilizadas, lo que facilita el desempaquetado de las entradas en la función de llamada del modelo. Pero, en nuestro caso, necesitamos las funciones no utilizadas ('imagen' en particular) para crear '`pixel_values`'.

Pasemos todo a `Trainer`.

Finalmente, definamos nuestro `Trainer`.

### Entrenamiento

### Evaluación

### Compartimos en el Hub